# MS-E2121 Exercise session 6

### Problem 6.1: Sensitivity analysis in the RHS

Consider the following LP and its optimal tableau below:

\begin{matrix}
		min         &  -2x_1  & - &   x_2  & + &  x_3  &       &    \\
		s.t.        &    x_1  & + &  2x_2  & + &  x_3  &  \leq & 8  \\
		            &   -x_1  & + &   x_2  & - & 2x_3  &  \leq & 4  \\
		            &   3x_1  & + &   x_2  &   &       &  \leq & 10 \\
            &    x_1, x_2, x_3  & \geq & 0   
\end{matrix}

<br/>

\begin{matrix}
      & | & \text{RHS} & | &    x_1 &   x_2 &   x_3  &   x_4  &   x_5  &   x_6  \\
 z=   & | &       -7.6 & | &      0 &     0 &   1.2  &   0.2  &     0  &   0.6  \\
 --   & | &        --- & | &    --- &   --- &   ---  &   ---  &    --- &    --- \\
x_1 = & | &        2.4 & | &      1 &     0 &  -0.2  &  -0.2  &     0  &   0.4  \\
x_2 = & | &        2.8 & | &      0 &     1 &   0.6  &   0.6  &     0  &  -0.2  \\
x_5 = & | &        3.6 & | &      0 &     0 &  -2.8  &  -0.8  &     1  &   0.6  \\
\end{matrix}

(a) If you were to choose between increasing in 1 unit the right hand side (RHS) of any constraints, which one would you choose, and why? What is the effect of the increase on the optimal cost? <br/>
<br/>
(b) Perform a sensitivity analysis on the model to discover what is the range of alteration in the RHS in which the same effect calculated in item (a) can be expected. *HINT*: JuMP 0.21.6 and above includes the function ``lp_sensitivity_report()`` that you can use to help performing the analysis.

In [ ]:
using JuMP, GLPK

Matrices A, b, and c:

In [ ]:
A = [ 1  2  1  1  0  0;
     -1  1 -2  0  1  0;
      3  1  0  0  0  1]

b = [8 ; 4 ; 10]
c = [-2 ; -1 ; 1 ; 0 ; 0 ; 0];

Using the optimal solution presented in the optimal tableau ($x_1 = 2.4$, $x_2 = 2.8$, $x_5 = 3.6$), we can check what is the optimal basis. Additionally, columns $A_4$, $A_5$, and $A_6$ stand for the slack variables.

In [ ]:
Bas = [1,2,5]
## Computing invB
B = A[:,Bas]
invB = inv(B);

Using the reduced costs from the optimal tableau, we can derive the duals $p_1, p_2, p_3$ as follows: <br/>
<br/>
$\bar{c}_j = c_j - c_B B^{-1} A_j = c_j - p^{\top}A_j$ <br/>

$0.2 = \bar{c}_4 = c_4 - \text{p}^{\top}\text{A}_4 = 0 - p_1 \Rightarrow p_1 = -0.2$ <br/>
$0 = \bar{c}_5 = c_5 - \text{p}^{\top}\text{A}_5 = 0 - p_2 \Rightarrow p_2 = 0$ <br/>
$0.6 = \bar{c}_6 = c_6 - \text{p}^{\top}\text{A}_6 = 0 - p_3 \Rightarrow p_3 = -0.6$ <br/>

Therefore, the effects on the optimal cost for increasing in 1 the RHS of the constraints $c_1, c_2, c_3$ is respectively $(-0.2,0,-0.6)$.
 
As shown in Lecture 6, slide 10, the bounds for the RHS alteration $\delta$ is: <br/>
<br/>
${max}_{j:g_{ji}>0} \left( -\frac{x_{\text{B(j)}}}{g_{ji}} \right) \leq \delta \leq {min}_{j:g_{ji}<0} \left( -\frac{x_{\text{B(j)}}}{g_{ji}} \right)$,

where $g_{ji}$ is the element on the $j$th row and $i$th column of $B^{-1}$.

In [ ]:
## Computing the range of changing for the constraint 1
bounds = -invB*b./invB[:,1]
pos = -invB*b./invB[:,1] .> 0
neg = -invB*b./invB[:,1] .< 0

lower_bound_1 = maximum(bounds[neg])
upper_bound_1 = minimum(bounds[pos]);

println("Changes in the RHS for c1 are constrained between [", round(lower_bound_1,digits=2) , "," ,
        round(upper_bound_1,digits=2), "]." )
println("b_1 is thus constrained between [", b[1]+round(lower_bound_1,digits=2) , "," ,
        b[1]+round(upper_bound_1,digits=2), "]." )

In [ ]:
## Computing the range of changing for the constraint 3
bounds = -invB*b./invB[:,3]
pos = -invB*b./invB[:,3] .> 0
neg = -invB*b./invB[:,3] .< 0

lower_bound_3 = maximum(bounds[neg])
upper_bound_3 = minimum(bounds[pos]);

println("Changes in the RHS for c3 are constrained between [", round(lower_bound_3,digits=2) , "," ,
        round(upper_bound_3,digits=3), "]." )
println("b_3 is thus constrained between [", b[3]+round(lower_bound_3,digits=2) , "," ,
        b[3]+round(upper_bound_3,digits=2), "]." )

We can do the same using JuMP:

In [ ]:
## Declaring the model
model = Model(GLPK.Optimizer);

In [ ]:
## Developing the LP

## Declaring variables
@variable(model, x1 >= 0)
@variable(model, x2 >= 0)
@variable(model, x3 >= 0)

## Declaring objective function
@objective(model, Min, -2x1 - x2 + x3)

## Declaring constraints
@constraint(model, con1, x1 + 2x2 + x3 <= 8)
@constraint(model, con2, -x1 + x2 -2x3 <= 4)
@constraint(model, con3, 3x1 + x2 <= 10);

In [ ]:
## Solving the LP
set_silent(model)
optimize!(model)
status = termination_status(model);
print(status)

In [ ]:
## Storing the optimal point
optimal_point = value.([x1 , x2 , x3]);

In [ ]:
## Storing the obj function at the original optimal point
obj = objective_value(model);

In [ ]:
## Storing dual relative to each constraint
duals = [dual(con1) , dual(con2) , dual(con3)];

The function ``lp_sensitivity_report()``, introduced in JuMP 0.21.6, returns objects mapping either the range in which a variable's coefficient in the OF can change without changing the optimal basis or the RHS sensitivity. More details at https://jump.dev/JuMP.jl/dev/reference/solutions/#JuMP.lp_sensitivity_report.

In [ ]:
report = lp_sensitivity_report(model);

If you name the constraint for the report you receive the range for the RHS in which the optimal solution is defined using the same basis.

In [ ]:
## Checking how the objective function can be altered with the RHS
for i in 1:3
    if duals[i] == 0
        println("Constraint $i is not binding.")
    else
        println("Constraint $i is binding, \nincreasing one unit on the RHS will change the optimal value by ", duals[i],
            " if b stays between [", b[i]+round(report[[con1,con2,con3][i]][1],digits = 2), " , ", b[i]+round(report[[con1,con2,con3][i]][2],digits = 2), "].")
    end
end

### Problem 6.4: Adding a new constraint

Given the LP below,

\begin{matrix}
		max         &   2x_1  & + &   x_2  &            \\
		s.t.        &   2x_1  & + &  2x_2  &  \leq & 9  \\
		            &   2x_1  & - &   x_2  &  \leq & 3  \\
		            &    x_1  &   &        &  \leq & 3  \\
		            &         &   &   x_2  &  \leq & 4  \\
                    &    x_1, x_2, x_3  & \geq & 0   
\end{matrix}

(a) Find the primal and dual optimal solutions. *HINT*: You can use complementary slackness to, once having the primal optimum, finding the dual optimal solution. <br/>
<br/>
(b) Suppose we add a new constraint $6x_1 - x_2 \leq 6$, classify the primal and dual former optimal points saying if they: (i) keep being optimal ; (ii) turn to be feasible but not optimal ; or (iii) turn infeasible.<br/>
<br/>
(c) Consider the new problem from item (b), find the new dual optimal point through one dual simplex iteration. After that, find the primal optimum using *complementary slackness*.

In [ ]:
using Plots
using LinearAlgebra

#### (a)

In [ ]:
A = [2.0 2.0 ; 2.0 -1.0 ; 1.0 0.0 ; 0.0 1.0]           # Constraints coefficients matrix
b = [9.0 ; 3.0 ; 3.0 ; 4.0]                            # RHS
c = [2.0 ; 1.0];                                       # OF's coefficients

In [ ]:
## Changing the problem to standard form
A_p = Matrix(hcat(A,I(4))) # I(4) is an identity matrix representing the slack variable coefficients
b_p = copy(b)              # RHS doesn't change
c_p = vcat(-c,zeros(4));   # Slack variables have objective coefficient zero

In [ ]:
# TODO: add your code here

In [ ]:
# TODO: add your code here

In [ ]:
# TODO: add your code here

#### (b)

#### (c)

New constraint: 6x1 - x2 <= 6

In [ ]:
## Writing the new constraint coefficient vector
# TODO: add your code here

New matrix A: \begin{bmatrix}
    \textbf{A}          & \textbf{0} \\
    \textbf{a}^\top _{m+1}   &  1
\end{bmatrix}

In [ ]:
## Updating A
# TODO: add your code here

New matrix B: 

$\bar{\text{B}} =$
\begin{bmatrix}
    \text{B}      &  \text{0} \\
    \text{a}^\top &  1
\end{bmatrix}

In [ ]:
## Updating B
# TODO: add your code here

New matrix invB: 

$\bar{\text{B}}^{\ -1} =$
\begin{bmatrix}
    \text{B}^{\ -1}           &  \text{0} \\
    -\text{a}^\top \text{B}^{\ -1}  &   1
\end{bmatrix}

In [ ]:
## Updating B⁻¹
# TODO: add your code here

In [ ]:
## Updating c
# TODO: add your code here

In [ ]:
## Updating the basis
# TODO: add your code here

In [ ]:
## Forming the reduced costs
# TODO: add your code here

In [ ]:
## Updating b
# TODO: add your code here

In [ ]:
## New solution (with x7 in the basis)
# TODO: add your code here

#### (c)

In [ ]:
# TODO: add your code here

In [ ]:
# TODO: add your code here

In [ ]:
# TODO: add your code here

In [ ]:
# TODO: add your code here

In [ ]:
# TODO: add your code here